 Import Required Libraries

In [1]:
import ee
import geemap
import pandas as pd
from IPython.display import Image

Authenticate Google Earth Engine

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AZEOvhU11ygwAJkBWj_jhpckJI0IeixutBEpag4kER6Z-ELD3iwmw3Dw61s

Successfully saved authorization token.


Import Data

In [34]:
#Alaska boundary
alaska_bound = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq('STUSPS','AK'))

In [40]:
#Import Digital Elevation Model
Dataset = ee.Image('USGS/3DEP/10m')
Elevation = Dataset.select('elevation')

#change alaska boundary to proper coordinate system
alaska_bound_reprojected = alaska_bound.geometry().transform('EPSG:4269')

#clip elevation to just alaska
Elevation_Alaska = Elevation.clip(alaska_bound_reprojected)

url = Elevation_Alaska.getThumbUrl({
    'min': 0, 'max': 2500 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display a thumbnail of elevation in France.
Image(url=url)
#geemap.show_image(Elevation)

In [134]:
Elevation_Alaska_reprojected = Elevation_Alaska.reproject(crs='WGS84')
#print(Elevation_Alaska.projection().getInfo())
#print(alaska_bound.projection().getInfo())
#create section of Ykdelta
lon = 62.83
lat = -153
poi = ee.Geometry.Point(lon,lat)
poi_reprojected = poi.transform('EPSG:4269')
box = [-165.3,62.2,-162.5,63.4]
roi =  ee.Geometry.Rectangle(box)

#get download link

link = Elevation_Alaska.getDownloadURL({
    'scale' : 70,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi
})
print(link)

url = Elevation_Alaska.getThumbUrl({
    'min': 0, 'max': 2500 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/e5de9a9e0a033b8ac0d6ed3b9f98779b-9c46cfb317ef20b520f687967caf8478:getPixels


In [84]:
#NDVI
Dataset = ee.ImageCollection('MODIS/061/MYD13Q1').filterDate('2018-01-01','2018-05-01')

# Define the bounding box for Alaska
earth_bbox = [-179,-89,179,89]
earth_geometry = ee.Geometry.Rectangle(earth_bbox)

alaska_bound_reprojected = alaska_bound.geometry().transform('SR-ORG:6974')

NDVI = Dataset.select('NDVI')
#NDVI_Alaska = NDVI.clip('alaska_bound_reprojected')
NDVI_Alaska = NDVI.max()#.clip(alaska_bound_reprojected)
url = NDVI_Alaska.getThumbUrl({
   'min': 0, 'max': 8000 ,'region': earth_geometry, 'dimensions':1024,
   'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
#Image(url=url)

In [87]:
Dataset = ee.ImageCollection('NOAA/CDR/AVHRR/NDVI/V5')
NDVI = Dataset.select('NDVI')
NDVI_test = NDVI.first()
url = NDVI_test.getThumbUrl({
   'min': 0, 'max': 8000, 'dimensions':1024,
   'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)